In [15]:
# Here we are using the same Skeleton of chatbot and improving it by adding loops and conditions to store the chat history.
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal
from langgraph.checkpoint.memory import InMemorySaver
import time

In [16]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str
    step3: str


In [17]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}


def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(30)  # Simulate long-running hang
    return {"step2": "done"}


def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"step3": 'done'}

In [18]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


In [ ]:
try:
    print("▶ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"},config={"configurable": {"thread_id": "thread-1"}}
    )
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")


In [ ]:
graph.get_state({"configurable": {"thread_id": "thread-1"}})

In [14]:
list(graph.get_state_history({"configurable": {"thread_id": "thread-1"}}))

[]

In [ ]:
#6.Rerun to show fault tolerant resume
final_state=graph.invoke(None ,{"configurable": {"thread_id": "thread-1"}})
print("final state: ", final_state)